# Signal processing course 2018/2019-1 @ ELTE
# Assignment 2
## 10.01.2018

## Task 3

In [ ]:
import numpy as np

import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [ ]:
# Initialize seaborn with custom settings
# Facecolor values from S. Conradi @S_Conradi/@profConradi
custom_settings = {
    'figure.facecolor': '#f4f0e8',
    'axes.facecolor': '#f4f0e8',
    'axes.edgecolor': '0.7',
    'axes.linewidth' : '2',
    'grid.color': '0.7',
    'grid.linestyle': '--',
    'grid.alpha': 0.6,
}
sns.set_theme(palette=sns.color_palette('deep', as_cmap=False),
              rc=custom_settings)

### Calculate Fourier coefficients

In [ ]:
from scipy.integrate import quad

In [ ]:
def fourier_coeffs(f, T, N, t0=0, *args):
    '''
    Calculates the Fourier coefficients :math:`a_k`, :math:`b_k` up to
    an arbitrary order :math:`N` for a given periodic signal using
    numerical integration.

    Parameters
    ----------
    f : callable
        The target periodic function
    T : float
        Period of the function
    N : int
        The highest order of harmonics to be considered in the
        approximation
    t0 : float
        The starting point of the period (default is 0)
    args : tuple, optional
        Extra arguments to pass to `f` inside the lambda func. Should
        not be confused with the optional `args` argument of
        `scipy.integrate.quad()` inside the decorator functions.
    '''
    # Employ the substitution defined above
    w0 = 2*np.pi / T

    def a_k(k):
        ''' Function to integrate for a_k '''
        func = lambda t: f(t, *args) * np.cos(k * t * w0)
        return quad(func, t0, t0 + T)[0] * 2 / T

    def b_k(k):
        ''' Function to integrate for b_k '''
        func = lambda t: f(t, *args) * np.sin(k * t * w0)
        return quad(func, t0, t0 + T)[0] * 2 / T

    a = np.array([a_k(k) for k in range(N)])
    b = np.array([b_k(k) for k in range(N)])  # note: `b_0` is always 0
    return a, b

### Calculate Fourier series partial sum

In [ ]:
def fourier_sin(t, a, b, N, T):
    '''
    Approximates a target periodic :math:`f_N(t)` function using the
    sine-cosine form of the Fourier series in the interval
    :math:`[t_0, t_0+T]`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    a : array-like
        List of :math:`a_k` coefficients
    b : array-like
        List of :math:`b_k` coefficients
    N : int
        The highest order of harmonics to be considered in the
        approximation
    T : float
        Period of the evaluated function
    '''
    # Do not enable larger values for N than the number of values in the
    # input `a` or `b` arrays. Also prevent N < 1 values.
    assert (N >= 1) | (len(a) >= 1), 'N should be larger than or equal to 1!'
    N = np.min((N, len(a)))

    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    # Employ the substitution P(t) = w0 * t
    P = 2*np.pi / T * t
    # Calculate the approximation up to an order N
    fN = np.zeros_like(t, dtype=float) + a[0]/2
    for k in range(1, N):
        fN += a[k] * np.cos(P * k) + b[k] * np.sin(P * k)

    if FCHECK:
        return fN[0]
    return fN

In [ ]:
def c_k(k, a, b):
    '''
    Calculates the conditional value of :math:`c_k` from its definition.

    Parameters
    ----------
    k : int
        The respective value of the :math:`k` running index
    a : array-like
        List of :math:`a_k` coefficients
    b : array-like
        List of :math:`b_k` coefficients
    '''
    if k > 0:
        return 0.5 * (a[k] - 1j * b[k])
    elif k == 0:
        return 0.5 * a[0]
    elif k < 0:
        return 0.5 * (a[k] + 1j * b[k])

In [ ]:
def fourier_exp(t, a, b, N, T):
    '''
    Approximates a target periodic :math:`f_N(t)` function using the
    exponential form of the Fourier series in the interval
    :math:`[t_0, t_0+T]`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    a : array-like
        List of :math:`a_k` coefficients
    b : array-like
        List of :math:`b_k` coefficients
    N : int
        The highest order of harmonics to be considered in the
        approximation
    T : float
        Period of the evaluated function
    '''
    # Do not enable larger values for N than the number of values in the
    # input `a` or `b` arrays. Also prevent N < 1 values.
    assert (N >= 1) | (len(a) >= 1), 'N should be larger than or equal to 1!'
    N = np.min((N, len(a)))

    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    # Employ the substitution P(t) = i * w0 * t
    P = 1j * 2*np.pi / T * t
    # Sum over 2N+1 number of `k` indices, running between ]-N -> N[
    fN = np.zeros_like(t, dtype=np.complex128)
    for k in range(-N+1, N):
        fN += c_k(k, a, b) * np.exp(P * k)

    if FCHECK:
        return fN[0]
    return fN

### Testing Fourier series on various functions
#### Common hyperparameters

In [ ]:
fs = 128  # Sampling rate for signals
N  = 50   # Highest order of harmonics considered

#### A problem with time

Choosing the appropriate time array is not entirely trivial. We have to account for the periodicity, the domain of the evaluated function/signal that it is periodic over and for an additional phase shift too.

1. Periodicity can be easily obtained by taking the time array modulo period $T$:
    $$t_{\mathrm{new}} = t \bmod T$$
2. Phase shift is also trivial as one need to simply shift the time array by some $\phi$ time shift:
    $$t_{\mathrm{new}} = t - \phi$$
3. However, transforming the time array to the correct periodicity domain needs a more thoughtful approach. Fortunately, we only need to consider two cases: *odd* and *even* functions. Even functions are symmetric about the $y$ axis, and their complete period extends over the interval $\left[ -T/2, T/2 \right]$. On the other hand, a complete period of odd functions extends over the interval $\left[ 0, T \right]$. Assuming either periodicity domains, we would want to shift and wrap $t$ into one of the domains. This can be achieved by first shifting $t$ so that its midpoint aligns with zero in case of even functions or $T/2$ in case of odd functions, and then applying the modulo operation to wrap it within the desired domain.

    Given that $t \bmod T$ is already center aligned on $T/2$ by default, we have no other work to do in case of odd functions. However, a direct modulo operation may not suffice for even functions for this very same reason, as it results in a domain of $\left[ 0, T \right]$. To shift this to the desired domain $\left[ -T/2, T/2 \right]$ for even functions, first we still need to shift the center from $T/2$ to zero by the operation $t + T/2$, then we would subtract $T/2$ after the modulo operation:
    $$t_{\mathrm{new}} = (t + T/2) \bmod T - T/2$$

So the final transformation would be as follows:

$$
    t_{\mathrm{new}} = (t - \phi \pm T_{0}) \bmod T - T_{0}
    \qquad
    T_{0}
    =
    \begin{cases}
        T/2 & \text{even} \\
        0   & \text{odd}
    \end{cases}
$$

#### A visual demo of the temporal transformations

In [ ]:
T = 2           # Period
t0, t1 = -T, T  # Range of the time array
# Treat `fs` as the number of samples per each period
t = np.linspace(t0, t1, int((t1-t0)/T * fs))

In [ ]:
nr, nc = 1, 4
fig, axes = plt.subplots(nr, nc, figsize=(nc*6, nr*6))
axes = axes.flatten()

trans = [
    r'$f(t) = t$',
    r'$f(t) = t\ \mathrm{mod}\ T$',
    r'$f(t) = \left( t \pm \frac{T}{2} \right)\ \mathrm{mod}\ T - \frac{T}{2}$',
    r'$f(t) = (t - \phi)\ \mathrm{mod}\ T$'
]
for i, ax in enumerate(axes):
    ax.set_xlabel('Time', fontsize=14)
    ax.set_ylabel('Amplitude', fontsize=14)
    # Set axis limits to be the same for each figure
    pad = 0.05 * (t1 - t0)
    ax.set_xlim(t0-pad, t1+pad)
    ax.set_ylim(t0-pad, t1+pad)
    # Indicate transformation on the plot
    ax.text(0.6, 0.1, f'{trans[i]}', fontsize=18,
            ha='center', va='center', transform=ax.transAxes)

# Step 1: The raw time array
ax = axes[0]
ax.plot(t, t, color='0.3', lw=4)
# Step 2: Periodize time
ax = axes[1]
ax.plot(t, t%T, color='0.3', lw=4)
# Step 3: Set periodicity domain
ax = axes[2]
ax.plot(t, t%T, color='0.3', ls='--', lw=4, alpha=0.3)
ax.plot(t, (t + T/2)%T - T/2, color='0.3', lw=4)
# Step 4: Apply phase shift
ax = axes[3]
phi = np.pi
ax.plot(t, t%T, color='0.3', ls='--', lw=4, alpha=0.3)
ax.plot(t, (t - phi)%T, color='0.3', lw=4)

plt.show()

#### 1) Pulse wave and square wave
The definition of a periodic pulse wave signal with $\tau$ duty cycle, $T$ period, $A$ amplitude and $\phi$ phase shift is
$$
    f(t)
    =
    \begin{cases}
        A  && \left| (t - \phi) \bmod T \right| \leq \tau \\
        -A && \left| (t - \phi) \bmod T \right| > \tau
    \end{cases}
$$

While the square wave is one of its special variant, where $\tau = T/2$.

In [ ]:
def f1(t, A, tau, T, T0=0, phi=0):
    '''
    Generate a pulse wave signal with a bandwidth of :math:`\tau`, a
    period of :math:`T`, an amplitude of ::math:`A` and a phase shift
    of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    A : float
        Amplitude of the output signal
    tau : float
        Duty cycle of the output signal
    T : float
        Period of the output signal
    T0 : float
        :math:`0` or :math:`T/2`, depending whether the signal is
        generated by an odd or an even function, respectively.
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The output signal evaluated at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    # Apply temporal transformations
    # 1. Phase shift is applied by `t - phi`
    # 2, Periodicity is applied by `(t - phi)%T`
    # 3. Periodic function domain is enforced by `(t - phi + T0)%T - T0`
    t = (t - phi + T0)%T - T0
    # Apply the definition of the signal
    x = np.abs(t)
    mask = (x <= tau)  # Use a mask to select correct values easily
    x[mask]  = A       # t <= tau --> f(t) = A
    x[~mask] = -A      # t >  tau --> f(t) = -A

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
A   = 1        # Amplitude of the pulse waves
tau = np.pi    # Duty cycle of the pulse waves
T   = 2*np.pi  # Period
T0  = 0        # Periodic domain shift amount
phi = 0        # Phase shift
# Sampling locations corresponding to the evaluated signal
t0, t1 = -T, T
t = np.linspace(t0, t1, endpoint=False, num=int((t1-t0)/T * fs))

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f1(t, A, tau, T, T0, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (A, tau, T, T0, phi)
a, b = fourier_coeffs(f1, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f1(t, A, tau, T, T0, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.1)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 2) Sawtooth wave

The generalized definition of a periodic sawtooth wave with $T$ period and $\phi$ phase shift is

$$
    f(t)
    =
    2 \left( \frac{t}{T} - \left\lfloor \frac{1}{2} + \frac{t}{T} \right\rfloor \right),
$$

where $\lfloor \dots \rfloor$ is the floor function.

In [ ]:
def f2(t, T, T0=0, phi=0):
    '''
    Generate a sawtooth wave signal with a period of :math:`T`, an
    amplitude of ::math:`1` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    T0 : float
        :math:`0` or :math:`T/2`, depending whether the signal is
        generated by an odd or an even function, respectively.
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    t = (t - phi + T0)%T - T0
    x = t
    # General form to generate sawtooth between -1 < f(t) < 1
    #x = 2 * (t/T - np.floor(1/2 + t/T))

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 2*np.pi  # Period
T0 = 0       # Periodic domain shift amount
phi = 0      # Phase shift
# Sampling locations corresponding to the evaluated signal
t0, t1 = -T, T
t = np.linspace(t0, t1, endpoint=False, num=int((t1-t0)/T * fs))

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f2(t, T, T0, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, T0, phi)
a, b = fourier_coeffs(f2, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f2(t, T, T0, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.15)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 3) "Periodic" absolute value function $-$ Triangle wave
In this section we consider a periodic, triangle wave-like function with a period $T$ and a phase shift $\phi$ defined by

$$
    f(t)
    =
    \left| (t - \phi) \bmod T \right|
    \qquad
    t \in \left[ -\pi, \pi \right],
    \qquad
    T = 2 \pi.
$$

In [ ]:
def f3(t, T, T0=0, phi=0):
    '''
    Generate a triangle wave signal with a period of :math:`T`, an
    amplitude of ::math:`T/2` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    T0 : float
        :math:`0` or :math:`T/2`, depending whether the signal is
        generated by an odd or an even function, respectively.
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    t = (t - phi + T0)%T - T0
    x = np.abs(t)

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 2*np.pi  # Period
T0 = np.pi   # Periodic domain shift amount
phi = 0      # Phase shift
# Sampling locations corresponding to the evaluated signal
t0, t1 = -T, T
t = np.linspace(t0, t1, endpoint=False, num=int((t1-t0)/T * fs))

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f3(t, T, T0, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, T0, phi)
a, b = fourier_coeffs(f3, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f3(t, T, T0, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.15)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 4) Shifted sawtooth
In this section we consider a periodic, sawtooth-like function with period $T$ and a phase shift $\phi$ defined by

$$
    f(t)
    =
    1 +  \left( t - \phi \right) \bmod T
    \qquad
    t \in \left[ -\pi, \pi \right],
    \qquad
    T = 2 \pi.
$$

In [ ]:
def f4(t, T, T0=0, phi=0):
    '''
    Generate a sawtooth-like signal with a period of :math:`T`, an
    amplitude of ::math:`1+T/2` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    T0 : float
        :math:`0` or :math:`T/2`, depending whether the signal is
        generated by an odd or an even function, respectively.
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    t = (t - phi + T0)%T - T0
    x = 1 + t

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 2*np.pi  # Period
T0 = np.pi   # Periodic domain shift amount
phi = 0      # Phase shift
# Sampling locations corresponding to the evaluated signal
t0, t1 = -T, T
t = np.linspace(t0, t1, endpoint=False, num=int((t1-t0)/T * fs))

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f4(t, T, T0, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, T0, phi)
a, b = fourier_coeffs(f4, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f4(t, T, T0, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.15)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 5) Periodic parabolae
In this section we consider a periodic function with period $T$ and a phase shift $\phi$ defined by

$$
    f(t)
    =
    \left[ \left( t - \phi \right) \bmod T \right]^{2}
    \qquad
    t \in \left[ -\pi, \pi \right],
    \qquad
    T = 2 \pi.
$$

In [ ]:
def f5(t, T, T0=0, phi=0):
    '''
    Generate a signal with a period of :math:`T`, an amplitude of
    ::math:`T^{2}/4` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    T0 : float
        :math:`0` or :math:`T/2`, depending whether the signal is
        generated by an odd or an even function, respectively.
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    t = (t - phi + T0)%T - T0
    x = t*t

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 2*np.pi  # Period
T0 = np.pi   # Periodic domain shift amount
phi = 0      # Phase shift
# Sampling locations corresponding to the evaluated signal
t0, t1 = -T, T
t = np.linspace(t0, t1, endpoint=False, num=int((t1-t0)/T * fs))

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f5(t, T, T0, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, T0, phi)
a, b = fourier_coeffs(f5, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f5(t, T, T0, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.15)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 6) Inverted periodic parabolae
In this section we consider a periodic function with period $T$ and a phase shift $\phi$ defined by

$$
    f(t)
    =
    \pi^2 - \left[ \left( t - \phi \right) \bmod T \right]^{2}
    \qquad
    t \in \left[ -\pi, \pi \right],
    \qquad
    T = 2 \pi.
$$

In [ ]:
def f6(t, T, T0=0, phi=0):
    '''
    Generate a signal with a period of :math:`T`, an amplitude of
    ::math:`T^{2}/4` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    T0 : float
        :math:`0` or :math:`T/2`, depending whether the signal is
        generated by an odd or an even function, respectively.
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    t = (t - phi + T0)%T - T0
    x = np.pi*np.pi - t*t

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 2*np.pi  # Period
T0 = np.pi   # Periodic domain shift amount
phi = 0      # Phase shift
# Sampling locations corresponding to the evaluated signal
t0, t1 = -T, T
t = np.linspace(t0, t1, endpoint=False, num=int((t1-t0)/T * fs))

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f6(t, T, T0, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, T0, phi)
a, b = fourier_coeffs(f6, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f6(t, T, T0, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.15)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 7) Inverted "smooth" sawtooth
In this section we consider a periodic, sawtooth-like function with period $T$ and a phase shift $\phi$ defined by

$$
    f(t_{c})
    =
    \left( t_{c} - 1 \right) \left( t_{c} - 3\right),
    \qquad
    t_{c} = \left( t - \phi \right) \bmod T,
    \qquad
    t \in \left[ -\pi, \pi \right],
    \qquad
    T = 2 \pi.
$$

In [ ]:
def f7(t, T, T0=0, phi=0):
    '''
    Generate a signal with a period of :math:`T`, and a phase shift of
    :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    T0 : float
        :math:`0` or :math:`T/2`, depending whether the signal is
        generated by an odd or an even function, respectively.
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    t = (t - phi + T0)%T - T0
    x = (t - 1) * (t - 3)

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 2*np.pi  # Period
T0 = np.pi   # Periodic domain shift amount
phi = 0      # Phase shift
# Sampling locations corresponding to the evaluated signal
t0, t1 = -T, T
t = np.linspace(t0, t1, endpoint=False, num=int((t1-t0)/T * fs))

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f7(t, T, T0, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, T0, phi)
a, b = fourier_coeffs(f7, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f7(t, T, T0, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.15)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 8) Yet another sawtooth-like wave
In this section we consider a periodic, sawtooth-like function with period $T$ and a phase shift $\phi$ defined by

$$
    f(t) = (t - \phi) \bmod T,
    \qquad
    t \in \left[ 0, 1 \right],
    \qquad
    T = 1.
$$

In [ ]:
def f8(t, T, T0=0, phi=0):
    '''
    Generate a signal with a period of :math:`T`, an amplitude of
    ::math:`T` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    T0 : float
        :math:`0` or :math:`T/2`, depending whether the signal is
        generated by an odd or an even function, respectively.
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    x = (t - phi + T0)%T - T0

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 1    # Period
T0 = 0   # Periodic domain shift amount
phi = 0  # Phase shift
# Sampling locations corresponding to the evaluated signal
t0, t1 = -T, T
t = np.linspace(t0, t1, endpoint=False, num=int((t1-t0)/T * fs))

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f8(t, T, T0, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, T0, phi)
a, b = fourier_coeffs(f8, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f8(t, T, T0, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.15)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 9) Rectangular function (also known as Heaviside Pi function, gate function, unit pulse, normalized boxcar function)
In this section we consider a periodic function with period $T$ and a phase shift $\phi$ defined by

$$
    f(t)
    =
    \begin{cases}
        0 && t \in \left[ -\pi, 0 \right] \\
        1 && t \in \left( 0, \pi \right]
    \end{cases}
    \qquad
    T = 2 \pi.
$$

In [ ]:
def f9(t, T, T0=0, phi=0):
    '''
    Generate a sqare wave-like signal with a period of :math:`T`, an
    amplitude of ::math:`T` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    T0 : float
        :math:`0` or :math:`T/2`, depending whether the signal is
        generated by an odd or an even function, respectively.
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    x = (t - phi - T/2)%T
    mask = (x <= T/2)
    x[mask]  = 0
    x[~mask] = 1

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 2*np.pi  # Period
T0 = 0       # Periodic domain shift amount
phi = 0      # Phase shift
# Sampling locations corresponding to the evaluated signal
t = np.linspace(-T, T, endpoint=False, num=fs)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f9(t, T, T0, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, T0, phi)
a, b = fourier_coeffs(f9, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f9(t, T, T0, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.15)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 10) An elongated square wave
In this section we consider a periodic function with period $T$ and a phase shift $\phi$ defined by

$$
    f(t)
    =
    \begin{cases}
        -1 && t \in \left[ -3, 0 \right] \\
        1  && t \in \left( 0, 3 \right]
    \end{cases}
    \qquad
    T = 6.
$$

In [ ]:
def f10(t, T, T0=0, phi=0):
    '''
    Generate a square wave-like signal with a period of :math:`T`, an
    amplitude of ::math:`T` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    T0 : float
        :math:`0` or :math:`T/2`, depending whether the signal is
        generated by an odd or an even function, respectively.
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    x = (t - phi - T/2)%T
    mask = (x <= T/2)
    x[mask]  = -1
    x[~mask] = 1

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 6    # Period
T0 = 0   # Periodic domain shift amount
phi = 0  # Phase shift
# Sampling locations corresponding to the evaluated signal
t = np.linspace(-T, T, endpoint=False, num=fs)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f10(t, T, T0, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, T0, phi)
a, b = fourier_coeffs(f10, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f10(t, T, T0, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.15)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 11) A discontinuous function
In this section we consider a periodic function with period $T$ and a phase shift $\phi$ defined by

$$
    f(t)
    =
    \begin{cases}
        1   && t \in \left[ -1, 0 \right) \\
        1/2 && t = 0  \\
        1   && t \in \left( 0, 1 \right]
    \end{cases}
    \qquad
    T = 2.
$$

In [ ]:
def f11(t, T, T0=0, phi=0):
    '''
    Generate a square wave-like signal with a period of :math:`T`, an
    amplitude of ::math:`T` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    T0 : float
        :math:`0` or :math:`T/2`, depending whether the signal is
        generated by an odd or an even function, respectively.
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    t = (t - phi - T/2)%T
    x = np.zeros_like(t)
    x[(t < T/2)] = 1
    x[(t == 0)]  = 1/2
    x[(t > T/2)] = t[(t > T/2)]

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 2    # Period
T0 = 0   # Periodic domain shift amount
phi = 0  # Phase shift
# Sampling locations corresponding to the evaluated signal
t = np.linspace(-T, T, endpoint=False, num=fs)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f11(t, T, T0, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, T0, phi)
a, b = fourier_coeffs(f11, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f11(t, T, T0, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.08)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 12) Sawtooth pulse
In this section we consider a periodic function with period $T$ and a phase shift $\phi$ defined by

$$
    f(t)
    =
    \begin{cases}
        0 && t \in \left[ -2, 0 \right] \\
        t && t \in \left( 0, 2 \right]
    \end{cases}
    \qquad
    T = 4.
$$

In [ ]:
def f12(t, T, T0=0, phi=0):
    '''
    Generate a square wave-like signal with a period of :math:`T`, an
    amplitude of ::math:`T` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    T0 : float
        :math:`0` or :math:`T/2`, depending whether the signal is
        generated by an odd or an even function, respectively.
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    t = (t - phi + T0)%T - T0
    x = np.zeros_like(t)
    mask = (t <= 0)
    x[mask]  = 0
    x[~mask] = t[~mask]

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 4    # Period
T0 = 2   # Periodic domain shift amount
phi = 0  # Phase shift
# Sampling locations corresponding to the evaluated signal
t = np.linspace(-T, T, endpoint=False, num=fs)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f12(t, T, T0, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, T0, phi)
a, b = fourier_coeffs(f12, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f12(t, T, T0, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.08)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 13) Sinusoidal pulse
In this section we consider a periodic function with period $T$ and a phase shift $\phi$ defined by

$$
    f(t)
    =
    \begin{cases}
        \sin \left( \frac{2 \pi t}{T} \right) && t \in \left[ 0, T/2 \right] \\
        0                                     && t \in \left( T/2, T \right]
    \end{cases}.
$$

In [ ]:
def f13(t, T, T0=0, phi=0):
    '''
    Generate a square wave-like signal with a period of :math:`T`, an
    amplitude of ::math:`T` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    T0 : float
        :math:`0` or :math:`T/2`, depending whether the signal is
        generated by an odd or an even function, respectively.
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    t = (t - phi + T0)%T - T0
    x = np.zeros_like(t)
    mask = (t <= T/2)
    x[mask]  = np.sin(2*np.pi*t[mask]/T)
    x[~mask] = 0

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 2    # Period
T0 = 0   # Periodic domain shift amount
phi = 0  # Phase shift
# Sampling locations corresponding to the evaluated signal
t = np.linspace(-T, T, endpoint=False, num=fs)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f13(t, T, T0, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, T0, phi)
a, b = fourier_coeffs(f13, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f13(t, T, T0, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.08)

ax.legend(loc='upper center', ncol=3)

plt.show()